In [1]:
import Bio
print(Bio.__version__)
import os
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Alphabet import generic_nucleotide
from Bio import Align
from Bio.SeqRecord import SeqRecord
from Bio.Seq import MutableSeq
from Bio.Alphabet import generic_dna
import re
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

offsetapplicablepair = 10000000
samenesslookahead = 2
variantnumber = 10
varheadoffset = 0
var_seq_record = 0
ref_gene_name = "BRCA1"
ref_seq_record = SeqIO.read(ref_gene_name+"-REF.fasta", "fasta")

dirpath = 'C:\\Users\\anjan.DESKTOP-PC5VIV8\\GPOC\\testfiles\\'

testfilecount = 0
testfilenames = []
#reporting section
reportlines = []
spcr = "|->| "
#strdiagnone = "Diagnosis: None"
strdiagnone = "---"

lstvarhits = []

1.77


In [2]:
reportlines.append("KAIGENIX CLINICAL MANAGEMENT - GENOMICS DEPT.".center(100))
strgeneline = "Genetic Variations Report (for "+ref_gene_name+" gene)\n"
reportlines.append(strgeneline.center(100))

reportlines.append("Details:")
reportlines.append("--------------------------------------------------------------------------------------------------------------------------")



In [3]:
'''
for root, dirs, files in os.walk(dirpath):
    for filename in files:
            print(filename)
            testfilecount+=1
            testfilenames.append(filename)

print(testfilecount)
for i in range(testfilecount):
    print(str(i) + " -- " + testfilenames[i])
'''

def getnetoffsetapplicable(start4digitnumber):
    netoffset = 0
    objectcount = len(lstvarhits)
    print(" in getnetoffsetapplicable function")
    for i in range(objectcount):
        
        tupobj = lstvarhits[i]
        print("***** next tuple *****")
        print("tupobj[0]: " + str(tupobj[0]))
        print("tupobj[1]: " + str(tupobj[1]))
        if start4digitnumber > tupobj[0]:
            netoffset += tupobj[1]
            print("netoffset temp = ", netoffset)
    
    print("netoffset total = ", netoffset)   
    return netoffset

In [4]:
'''
for seq_record in SeqIO.parse("BRCA1-REF.fasta", "fasta"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))
'''
   
#ref_seq_record = SeqIO.read("BRCA1-REF.fasta", "fasta")


'\nfor seq_record in SeqIO.parse("BRCA1-REF.fasta", "fasta"):\n    print(seq_record.id)\n    print(repr(seq_record.seq))\n    print(len(seq_record))\n'

In [5]:
'''
if variantnumber == 1:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR01.fasta", "fasta")
if variantnumber == 2:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR02.fasta", "fasta")
if variantnumber == 3:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR03.fasta", "fasta")
if variantnumber == 4:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR04.fasta", "fasta")
if variantnumber == 5:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR05.fasta", "fasta")
if variantnumber == 6:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR06.fasta", "fasta")
if variantnumber == 7:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR07.fasta", "fasta")
if variantnumber == 8:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR08.fasta", "fasta")
if variantnumber == 9:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR09.fasta", "fasta")
if variantnumber == 10:
    var_seq_record = SeqIO.read("P001-BRCA1-VAR10.fasta", "fasta") 
'''

'\nif variantnumber == 1:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR01.fasta", "fasta")\nif variantnumber == 2:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR02.fasta", "fasta")\nif variantnumber == 3:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR03.fasta", "fasta")\nif variantnumber == 4:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR04.fasta", "fasta")\nif variantnumber == 5:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR05.fasta", "fasta")\nif variantnumber == 6:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR06.fasta", "fasta")\nif variantnumber == 7:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR07.fasta", "fasta")\nif variantnumber == 8:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR08.fasta", "fasta")\nif variantnumber == 9:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR09.fasta", "fasta")\nif variantnumber == 10:\n    var_seq_record = SeqIO.read("P001-BRCA1-VAR10.fasta", "fasta") \n'

In [6]:
'''
print(var_seq_record.seq[:100])
print(ref_seq_record.seq[:100])
'''

'\nprint(var_seq_record.seq[:100])\nprint(ref_seq_record.seq[:100])\n'

In [7]:
def longform(strdiagnosis):
    if strdiagnosis == "BB": return "BENIGN"
    if strdiagnosis == "LB": return "LIKELY BENIGN"
    if strdiagnosis == "PP": return "PATHOLOGICAL"
    if strdiagnosis == "LP": return "LIKELY PATHOLOGICAL"
    if strdiagnosis == "NN": return "NOT KNOWN"
    return "NOT KNOWN"

In [8]:
#  sample -- c.0010C>T##BB 
def verifyvarient1present(variantrule):
    print("in rule verification 1----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record
    
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    
    '''
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)
    
    strbeforesubstitution=variantrule[6]
    straftersubstitution=variantrule[8]
    print(strbeforesubstitution)
    print(straftersubstitution)
    print('substitution confirmation 1')
    strdiagnosis = variantrule[11] + variantrule[12]
    print(strdiagnosis)
    print('var head offset 1--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    
    print(var_seq_record.seq[start4digitnumber + varheadoffsetTemp])
    print(ref_seq_record.seq[start4digitnumber])
    print('substitution confirmation 2')
    if var_seq_record.seq[start4digitnumber + varheadoffsetTemp] == straftersubstitution:
        if ref_seq_record.seq[start4digitnumber] == strbeforesubstitution:
            reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT  !!!!!!!!!!!!!!  ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
            print(variantrule + "      PRESENT                               ******************")
        else:
            reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )
        
    # no change to varheadoffset

In [9]:
#  sample -- c.0005_0008del##LB
def verifyvarient2present(variantrule):
    print("in rule verification 2----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record
    
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    end4digitnumber = int(variantrule[7])*1000 + int(variantrule[8])*100 + int(variantrule[9])*10 + int(variantrule[10])*1
    print(end4digitnumber)
    
    '''
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)
    
    strdiagnosis = variantrule[16] + variantrule[17]
    print(strdiagnosis)
    print('var head offset 2--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    
    difflength = end4digitnumber - start4digitnumber +1
    print(difflength)
    deleted = False
    nextdigitssame = True
    
    strvarsegment = ""
    strrefsegment = ""
    for i in range(difflength):
        print('var ' + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i])
        print('ref ' + ref_seq_record.seq[start4digitnumber+i])
        strvarsegment = strvarsegment + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i]
        strrefsegment = strrefsegment + ref_seq_record.seq[start4digitnumber+i]
        if var_seq_record.seq[start4digitnumber +varheadoffsetTemp +i] != ref_seq_record.seq[start4digitnumber+i]:
            deleted = True
    print('strvarsegment ' + strvarsegment)
    print('strrefsegment ' + strrefsegment)
    print('deletion comfirmation above, NOT same is good')
    
    strnextvarsegment = ""
    strnextrefsegment = ""
    for j in range(samenesslookahead):
        print('var next ' + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +j])
        print('ref next ' + ref_seq_record.seq[end4digitnumber+1+j])
        strnextvarsegment = strnextvarsegment + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +j]
        strnextrefsegment = strnextrefsegment + ref_seq_record.seq[end4digitnumber+1+j]
        if var_seq_record.seq[start4digitnumber +varheadoffsetTemp +j] != ref_seq_record.seq[end4digitnumber+1+j]:
            nextdigitssame = False
    print('strnextvarsegment ' + strnextvarsegment)
    print('strnextrefsegment ' + strnextrefsegment)  
    print('same segment confirmation above, same is good')

    
    if deleted == True and nextdigitssame == True:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT  !!!!!!!!!!!!!!  ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
        print(variantrule + "      PRESENT                              ******************")
        '''
        varheadoffset = varheadoffset - difflength
        print('new var head offset --' + str(varheadoffset))
        offsetapplicablepair = min(offsetapplicablepair, start4digitnumber)
        print('new offsetapplicablepair == ' + str(offsetapplicablepair))
        '''
        varoffset = 0 - difflength
        tup = (start4digitnumber, varoffset)
        lstvarhits.append(tup)
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )
    

In [10]:
# sample -- c.0009_0012delTCCT##PP
def verifyvarient3present(variantrule):
    print("in rule verification 3----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record
    
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    end4digitnumber = int(variantrule[7])*1000 + int(variantrule[8])*100 + int(variantrule[9])*10 + int(variantrule[10])*1
    print(end4digitnumber)
    
    '''
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)
    
        #get the string
    strdel=''
    sequencepointer = 14
    while variantrule[sequencepointer] != '#':
        strdel = strdel + variantrule[sequencepointer]
        sequencepointer += 1
    print(strdel)
    
    sequencepointer += 2
    strdiagnosis = variantrule[sequencepointer] + variantrule[sequencepointer+1]
    print(strdiagnosis)
    print('var head offset 3--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    
    difflength = end4digitnumber - start4digitnumber +1
    print(difflength)
    deleted = False
    nextdigitssame = True
    rightseqdeleted = True
    
    strvarsegment = ""
    strrefsegment = ""
    for i in range(difflength):
        print('var ' + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i])
        print('ref ' + ref_seq_record.seq[start4digitnumber+i])
        strvarsegment = strvarsegment + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i]
        strrefsegment = strrefsegment + ref_seq_record.seq[start4digitnumber+i]
        if var_seq_record.seq[start4digitnumber +varheadoffsetTemp +i] != ref_seq_record.seq[start4digitnumber+i]:
            deleted = True
    print('strvarsegment ' + strvarsegment)
    print('strrefsegment ' + strrefsegment)
    print('deletion comfirmation above, NOT same is good')
    
    strnextvarsegment = ""
    strnextrefsegment = ""
    for j in range(samenesslookahead):
        print('var next ' + var_seq_record.seq[start4digitnumber + varheadoffset +j])
        print('ref next ' + ref_seq_record.seq[end4digitnumber+1+j])
        strnextvarsegment = strnextvarsegment + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +j]
        strnextrefsegment = strnextrefsegment + ref_seq_record.seq[end4digitnumber+1+j]
        if var_seq_record.seq[start4digitnumber +varheadoffsetTemp +j] != ref_seq_record.seq[end4digitnumber+1+j]:
            nextdigitssame = False
    print('strnextvarsegment ' + strnextvarsegment)
    print('strnextrefsegment ' + strnextrefsegment)    
    print('same segment comfirmation above, same is good')
    
    for k in range(difflength):
        print('strdel ' + strdel[k])
        print('ref ' + ref_seq_record.seq[start4digitnumber+k])
        if strdel[k] != ref_seq_record.seq[start4digitnumber+k]:
            rightseqdeleted = False
    print('right seq deleted comfirmation above, same is good')
    
    if deleted == True and rightseqdeleted == True and nextdigitssame == True:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT  !!!!!!!!!!!!!!  ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
        print(variantrule + "      PRESENT                                ******************")
        '''
        varheadoffset = varheadoffset - difflength
        print('new var head offset --' + str(varheadoffset))
        offsetapplicablepair = min(offsetapplicablepair, start4digitnumber)
        print('new offsetapplicablepair == ' + str(offsetapplicablepair))
        '''
        varoffset = 0 - difflength
        tup = (start4digitnumber, varoffset)
        lstvarhits.append(tup)
        
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )
    

In [11]:
# sample -- c.0009_0012dup##LP
def verifyvarient4present(variantrule):
    print("in rule verification 4----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record

        
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    end4digitnumber = int(variantrule[7])*1000 + int(variantrule[8])*100 + int(variantrule[9])*10 + int(variantrule[10])*1
    print(end4digitnumber)
    
    ''' 
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)
    
    strdiagnosis = variantrule[16] + variantrule[17]
    print(strdiagnosis)
    print('var head offset 4--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    #varheadoffset = 412
    
    difflength = end4digitnumber - start4digitnumber +1
    print(difflength)
    duplicated = True
    nextdigitssame = True
   
    
    strvarsegment = ""
    strrefsegment = ""
    for i in range(difflength):
        print('var ' + var_seq_record.seq[end4digitnumber+1 + varheadoffsetTemp +i])
        print('ref ' + ref_seq_record.seq[start4digitnumber+i])
        strvarsegment = strvarsegment + var_seq_record.seq[end4digitnumber +1 + varheadoffsetTemp +i]
        strrefsegment = strrefsegment + ref_seq_record.seq[start4digitnumber+i]
        if var_seq_record.seq[end4digitnumber+1 +varheadoffsetTemp +i] != ref_seq_record.seq[start4digitnumber+i]:
            duplicated = False
    print('strvarsegment ' + strvarsegment)
    print('strrefsegment ' + strrefsegment)
    print('duplication comfirmation above, same is good')
    
    strnextvarsegment = ""
    strnextrefsegment = ""
    for j in range(samenesslookahead):
        print('var next ' + var_seq_record.seq[end4digitnumber+difflength+1 + varheadoffsetTemp +j])
        print('ref next ' + ref_seq_record.seq[end4digitnumber+1+j])
        strnextvarsegment = strnextvarsegment + var_seq_record.seq[end4digitnumber+1+ difflength + varheadoffsetTemp +j]
        strnextrefsegment = strnextrefsegment + ref_seq_record.seq[end4digitnumber+1+j]
        if var_seq_record.seq[end4digitnumber+1+difflength +varheadoffsetTemp +j] != ref_seq_record.seq[end4digitnumber+1+j]:
            nextdigitssame = False
    print('strnextvarsegment ' + strnextvarsegment)
    print('strnextrefsegment ' + strnextrefsegment)  
    print('same segment comfirmation above, same is good')
    
    
    if duplicated == True and nextdigitssame == True:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT  !!!!!!!!!!!!!!  ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
        print(variantrule + "      PRESENT                                  ******************")
        '''
        varheadoffset = varheadoffset + difflength
        print('new var head offset --' + str(varheadoffset))
        offsetapplicablepair = min(offsetapplicablepair, start4digitnumber)
        print('new offsetapplicablepair == ' + str(offsetapplicablepair))
        '''
        varoffset = 0 + difflength
        tup = (start4digitnumber, varoffset)
        lstvarhits.append(tup)
        
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )
    
    

In [12]:
# sample -- c.0014_0017dupGACG##NN
def verifyvarient5present(variantrule):
    print("in rule verification 5----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record


    
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    end4digitnumber = int(variantrule[7])*1000 + int(variantrule[8])*100 + int(variantrule[9])*10 + int(variantrule[10])*1
    print(end4digitnumber)
    
    '''
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)
    
    strdup=''
    sequencepointer = 14
    while variantrule[sequencepointer] != '#':
        strdup = strdup + variantrule[sequencepointer]
        sequencepointer += 1
    print(strdup)
    sequencepointer += 2
    strdiagnosis = variantrule[sequencepointer] + variantrule[sequencepointer+1]
    print(strdiagnosis)
    
    print('var head offset 5--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    
    difflength = end4digitnumber - start4digitnumber +1
    print(difflength)
    duplicated = True
    nextdigitssame = True
    correctduplicateseq = True
    
    strvarsegment = ""
    strrefsegment = ""
    for i in range(difflength):
        print("var pos = ", str(end4digitnumber+1 + varheadoffsetTemp +i))
        print('var ' + var_seq_record.seq[end4digitnumber+1 + varheadoffsetTemp +i])
        print("ref pos = ", str(start4digitnumber+i))
        print('ref ' + ref_seq_record.seq[start4digitnumber+i])
        strvarsegment = strvarsegment + var_seq_record.seq[end4digitnumber +1 + varheadoffsetTemp +i]
        strrefsegment = strrefsegment + ref_seq_record.seq[start4digitnumber+i]
        if var_seq_record.seq[end4digitnumber+1 +varheadoffsetTemp +i] != ref_seq_record.seq[start4digitnumber+i]:
            duplicated = False
    print('strvarsegment ' + strvarsegment)
    print('strrefsegment ' + strrefsegment)
    print('duplication comfirmation above, same is good')
    
    strnextvarsegment = ""
    strnextrefsegment = ""
    for j in range(samenesslookahead):
        print('var next ' + var_seq_record.seq[end4digitnumber+difflength+1 + varheadoffsetTemp +j])
        print('ref next ' + ref_seq_record.seq[end4digitnumber+1+j])
        strnextvarsegment = strnextvarsegment + var_seq_record.seq[end4digitnumber+1+ difflength + varheadoffsetTemp +j]
        strnextrefsegment = strnextrefsegment + ref_seq_record.seq[end4digitnumber+1+j]
        if var_seq_record.seq[end4digitnumber+1+difflength +varheadoffsetTemp +j] != ref_seq_record.seq[end4digitnumber+1+j]:
            nextdigitssame = False
    print('strnextvarsegment ' + strnextvarsegment)
    print('strnextrefsegment ' + strnextrefsegment) 
    print('same segment comfirmation above, same is good')
    
    for k in range(difflength):
        print('strdup ' + strdup[k])
        print('ref ' + ref_seq_record.seq[start4digitnumber+k])
        if strdup[k] != ref_seq_record.seq[start4digitnumber+k]:
            correctduplicateseq = False
    print('correct duplicate seq comfirmation above, same is good')
            
            
    if duplicated == True and nextdigitssame == True and correctduplicateseq == True:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT  !!!!!!!!!!!!!!  ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
        print(variantrule + "      PRESENT                                    ******************")
        
        '''
        varheadoffset = varheadoffset + difflength
        print('new var head offset --' + str(varheadoffset))
        offsetapplicablepair = min(offsetapplicablepair, start4digitnumber)
        print('new offsetapplicablepair == ' + str(offsetapplicablepair))
        '''
        varoffset = 0 + difflength
        tup = (start4digitnumber, varoffset)
        lstvarhits.append(tup)
        
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )
    #varheadoffset = 512
    

In [13]:
# sample -- c.0012_0013insATATAT##BB
def verifyvarient6present(variantrule):
    print("in rule verification 6----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record
    
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    end4digitnumber = int(variantrule[7])*1000 + int(variantrule[8])*100 + int(variantrule[9])*10 + int(variantrule[10])*1
    print(end4digitnumber)
    
    '''
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)
    print("||||||||varheadoffsetTemp ="+ str(varheadoffsetTemp)) 
    
    strins=''
    sequencepointer = 14
    while variantrule[sequencepointer] != '#':
        strins = strins + variantrule[sequencepointer]
        sequencepointer += 1
    print("strins: " + strins)
    lengthstrins = len(strins)
    sequencepointer += 2
    strdiagnosis = variantrule[sequencepointer] + variantrule[sequencepointer+1]
    print(strdiagnosis)
    
    print('var head offset 6--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    
    difflength = end4digitnumber - start4digitnumber +1
    print("difflength = ", difflength)
    
    nextdigitssame = True
    correctinsertedseq = True
    
    '''
    strvarsegment = ""
    strrefsegment = ""
    for i in range(lengthstrins):
        print('var ' + var_seq_record.seq[start4digitnumber+1 + varheadoffsetTemp +i])
        print(start4digitnumber+1 + varheadoffsetTemp +i)
        print('ref ' + ref_seq_record.seq[start4digitnumber+1+i])
        print(start4digitnumber+1+i)
        strvarsegment = strvarsegment + var_seq_record.seq[start4digitnumber +1 + varheadoffsetTemp +i]
        strrefsegment = strrefsegment + ref_seq_record.seq[start4digitnumber+1+i]
        if var_seq_record.seq[start4digitnumber+1 +varheadoffsetTemp +i] != ref_seq_record.seq[start4digitnumber+1+i]:
            duplicated = False
    print('strvarsegment ' + strvarsegment)
    print('strrefsegment ' + strrefsegment)
    
    print('duplicated? why not inserted? comfirmation above, NOT same is good')
    '''
    
    strnextvarsegment = ""
    strnextrefsegment = ""
    for j in range(samenesslookahead):
        print('var next ' + var_seq_record.seq[end4digitnumber+lengthstrins + varheadoffsetTemp +j])
        print('ref next ' + ref_seq_record.seq[end4digitnumber+j])
        strnextvarsegment = strnextvarsegment + var_seq_record.seq[end4digitnumber+ lengthstrins + varheadoffsetTemp +j]
        strnextrefsegment = strnextrefsegment + ref_seq_record.seq[end4digitnumber+j]
        if var_seq_record.seq[end4digitnumber+lengthstrins +varheadoffsetTemp +j] != ref_seq_record.seq[end4digitnumber+j]:
            nextdigitssame = False
    print('strnextvarsegment ' + strnextvarsegment)
    print('strnextrefsegment ' + strnextrefsegment)    
    print('same segment comfirmation above, same is good')
    
    for k in range(lengthstrins):
        print('strins ' + strins[k])
        print('var ' + var_seq_record.seq[start4digitnumber+varheadoffsetTemp+1+k])
        if strins[k] != var_seq_record.seq[start4digitnumber+varheadoffsetTemp+1+k]:
            correctinsertedseq = False
    
    print('correct inserted seq comfirmation above, same is good')
            
            
    if correctinsertedseq == True and nextdigitssame == True:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT  !!!!!!!!!!!!!!  ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
        print(variantrule + "      PRESENT                                    ******************")
        
        '''
        varheadoffset = varheadoffset + lengthstrins
        print('new var head offset --' + str(varheadoffset))
        offsetapplicablepair = min(offsetapplicablepair, start4digitnumber)
        print('new offsetapplicablepair == ' + str(offsetapplicablepair))
        '''
        varoffset = 0 + lengthstrins
        tup = (start4digitnumber, varoffset)
        lstvarhits.append(tup)
        
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )
    #varheadoffset = 512
    
    

In [14]:
# sample -- c.0009_0010delinsGGAA##LP
def verifyvarient7present(variantrule):
    print("in rule verification 7----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record
    
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    end4digitnumber = int(variantrule[7])*1000 + int(variantrule[8])*100 + int(variantrule[9])*10 + int(variantrule[10])*1
    print(end4digitnumber)
    
    '''
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)
    
    strdelins=''
    sequencepointer = 17
    while variantrule[sequencepointer] != '#':
        strdelins = strdelins + variantrule[sequencepointer]
        sequencepointer += 1
    print(strdelins)
    lengthstrdelins = len(strdelins)
    
    sequencepointer += 2
    strdiagnosis = variantrule[sequencepointer] + variantrule[sequencepointer+1]
    print(strdiagnosis)
    
    print('var head offset 7--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    #varheadoffset = 712
    
    difflength = end4digitnumber - start4digitnumber +1
    print(difflength)
    deleted = False
    inserted = True
    nextdigitssame = True
    correctinsertedseq = True
    
    strvarsegment = ""
    strrefsegment = ""
    for i in range(difflength):
        print('var ' + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i])
        print('ref ' + ref_seq_record.seq[start4digitnumber+i])
        strvarsegment = strvarsegment + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i]
        strrefsegment = strrefsegment + ref_seq_record.seq[start4digitnumber+i]
        if var_seq_record.seq[start4digitnumber +varheadoffsetTemp +i] != ref_seq_record.seq[start4digitnumber+i]:
            deleted = True
    print('strvarsegment ' + strvarsegment)
    print('strrefsegment ' + strrefsegment)
    
    print('deleted comfirmation above, not same is true')
    
    strnextvarsegment = ""
    strnextrefsegment = ""
    for j in range(samenesslookahead):
        print('var next ' + var_seq_record.seq[start4digitnumber+lengthstrdelins + varheadoffsetTemp +j])
        print('ref next ' + ref_seq_record.seq[end4digitnumber+1+j])
        strnextvarsegment = strnextvarsegment + var_seq_record.seq[start4digitnumber+ lengthstrdelins + varheadoffsetTemp +j]
        strnextrefsegment = strnextrefsegment + ref_seq_record.seq[end4digitnumber+1+j]
        if var_seq_record.seq[start4digitnumber+lengthstrdelins +varheadoffsetTemp +j] != ref_seq_record.seq[end4digitnumber+1+j]:
            nextdigitssame = False
    print('strnextvarsegment ' + strnextvarsegment)
    print('strnextrefsegment ' + strnextrefsegment) 
    
    print('same segment comfirmation above, same is good')
    
    for k in range(lengthstrdelins):
        print('strdelins ' + strdelins[k])
        print('var ' + var_seq_record.seq[start4digitnumber+varheadoffsetTemp+k])
        if strdelins[k] != var_seq_record.seq[start4digitnumber+varheadoffsetTemp+k]:
            correctinsertedseq = False
    
    print('correct inserted seq comfirmation above, same is good')
    
    if deleted == True and nextdigitssame == True and correctinsertedseq == True:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT  !!!!!!!!!!!!!!  ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
        print(variantrule + "      PRESENT                                               ******************")
        
        '''
        varheadoffset = varheadoffset - difflength + lengthstrdelins
        print('new var head offset --' + str(varheadoffset))
        offsetapplicablepair = min(offsetapplicablepair, start4digitnumber)
        print('new offsetapplicablepair == ' + str(offsetapplicablepair))
        '''
        varoffset = 0 - difflength + lengthstrdelins
        tup = (start4digitnumber, varoffset)
        lstvarhits.append(tup)
        
        
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )
    #varheadoffset = 512
   
    

In [15]:
# sample -- c.0009_0010delTCinsGGAA##LP
def verifyvarient8present(variantrule):
    print("in rule verification 8 ----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record
    
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    end4digitnumber = int(variantrule[7])*1000 + int(variantrule[8])*100 + int(variantrule[9])*10 + int(variantrule[10])*1
    print(end4digitnumber)
    
    '''
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)
    
    strdel=''
    sequencepointer = 14
    char = variantrule[sequencepointer]
    print(char)
    while ((char != 'I') and (char != 'i')):
        strdel = strdel + char
        sequencepointer += 1
        char = variantrule[sequencepointer]
        print(char)
    print(strdel)
    lengthstrdel = len(strdel)
    
    strins=''
    sequencepointer += 3
    while variantrule[sequencepointer] != '#':
        strins = strins + variantrule[sequencepointer]
        sequencepointer += 1
    print(strins)
    lengthstrins = len(strins)
    
    sequencepointer += 2
    strdiagnosis = variantrule[sequencepointer] + variantrule[sequencepointer+1]
    print(strdiagnosis)
    
    print('var head offset 8--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    #varheadoffset = 712
    
    difflength = end4digitnumber - start4digitnumber +1
    print(difflength)
    deleted = False
    inserted = True
    nextdigitssame = True
    correctdeletedseq = True
    correctinsertedseq = True
    
    strvarsegment = ""
    strrefsegment = ""
    for i in range(difflength):
        print('var ' + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i])
        print(start4digitnumber + varheadoffsetTemp +i)
        print('ref ' + ref_seq_record.seq[start4digitnumber+i])
        print(start4digitnumber+i)
        strvarsegment = strvarsegment + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i]
        strrefsegment = strrefsegment + ref_seq_record.seq[start4digitnumber+i]
        if var_seq_record.seq[start4digitnumber +varheadoffsetTemp +i] != ref_seq_record.seq[start4digitnumber+i]:
            deleted = True
    print('strvarsegment ' + strvarsegment)
    print('strrefsegment ' + strrefsegment)
    
    print('deleted comfirmation above. not same is good')
    
    strnextvarsegment = ""
    strnextrefsegment = ""
    for j in range(samenesslookahead):
        print('var next ' + var_seq_record.seq[start4digitnumber+lengthstrins + varheadoffsetTemp +j])
        print('ref next ' + ref_seq_record.seq[end4digitnumber+1+j])
        strnextvarsegment = strnextvarsegment + var_seq_record.seq[start4digitnumber+ lengthstrins + varheadoffsetTemp +j]
        strnextrefsegment = strnextrefsegment + ref_seq_record.seq[end4digitnumber+1+j]
        if var_seq_record.seq[start4digitnumber+lengthstrins +varheadoffsetTemp +j] != ref_seq_record.seq[end4digitnumber+1+j]:
            nextdigitssame = False
    print('strnextvarsegment ' + strnextvarsegment)
    print('strnextrefsegment ' + strnextrefsegment) 
    
    print('same segment comfirmation above, same is good')
    
    for k in range(lengthstrdel):
        print('strdel ' + strdel[k])
        print('ref ' + ref_seq_record.seq[start4digitnumber+k])
        if strdel[k] != ref_seq_record.seq[start4digitnumber+k]:
            correctdeletedseq = False
            
    print('correct deleted seq comfirmation above, same is good')
    
    for l in range(lengthstrins):
        print('strins ' + strins[l])
        print('var ' + var_seq_record.seq[start4digitnumber+varheadoffsetTemp+l])
        if strins[l] != var_seq_record.seq[start4digitnumber+varheadoffsetTemp+l]:
            correctinsertedseq = False
    print('correct inserted seq comfirmation above, same is good')
    
    if deleted == True and nextdigitssame == True and correctinsertedseq == True and correctdeletedseq == True:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT  !!!!!!!!!!!!!!  ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
        print(variantrule + "      PRESENT                                       ******************")
        
        '''
        varheadoffset = varheadoffset + difflength
        print('new var head offset --' + str(varheadoffset))
        offsetapplicablepair = min(offsetapplicablepair, start4digitnumber)
        print('new offsetapplicablepair == ' + str(offsetapplicablepair))
        '''
        
        varoffset = 0 - lengthstrdel + lengthstrins
        tup = (start4digitnumber, varoffset)
        lstvarhits.append(tup)
        
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr+ "Absent".ljust(30)+ spcr + strdiagnone )
    #varheadoffset = 512
   
    

In [16]:
# sample -- c.0021_0039inv##PP
def verifyvarient9present(variantrule):
    #c.0021_0039inv##PP
    print("in rule verification 9 ----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record
    #feature_seq = example_parent[example_feature.location.start:example_feature.location.end].reverse_complement()
    
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    end4digitnumber = int(variantrule[7])*1000 + int(variantrule[8])*100 + int(variantrule[9])*10 + int(variantrule[10])*1
    print(end4digitnumber)
    
    '''
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)

    sequencepointer = 16
    strdiagnosis = variantrule[sequencepointer] + variantrule[sequencepointer+1]
    print(strdiagnosis)
    
    print('var head offset 9--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    
    
    difflength = end4digitnumber - start4digitnumber +1
    print(difflength)
    
    isinverted = True
    
    nextdigitssame = True

    invertedseq = var_seq_record.seq[start4digitnumber+varheadoffsetTemp:end4digitnumber+varheadoffsetTemp+1]
    
    correspondingrefseq = ref_seq_record.seq[start4digitnumber:end4digitnumber+1]
    
    
    print(invertedseq)
    originalseqfromvarseq = invertedseq.reverse_complement()
    print(originalseqfromvarseq)
    
    print(correspondingrefseq)
    
    
    strvarsegment = ""
    strrefsegment = ""
    for i in range(difflength):
        print('var ' + originalseqfromvarseq[i])
        print('ref ' + correspondingrefseq[i])
        if originalseqfromvarseq[i] != correspondingrefseq[i]:
            isinverted = False
    print('strvarsegment which is inverted ' + invertedseq)
    print('strvarsegment which is inverted back ' + originalseqfromvarseq)
    print('strrefsegment ' + correspondingrefseq)
    
    print('inversion reverse complement comfirmation above, same [2 and 3] is good')
    
    strnextvarsegment = ""
    strnextrefsegment = ""
    for j in range(samenesslookahead):
        print('var next ' + var_seq_record.seq[start4digitnumber+difflength + varheadoffsetTemp +j])
        print('ref next ' + ref_seq_record.seq[end4digitnumber+1+j])
        strnextvarsegment = strnextvarsegment + var_seq_record.seq[start4digitnumber+ difflength + varheadoffsetTemp +j]
        strnextrefsegment = strnextrefsegment + ref_seq_record.seq[end4digitnumber+1+j]
        if var_seq_record.seq[start4digitnumber+difflength +varheadoffsetTemp +j] != ref_seq_record.seq[end4digitnumber+1+j]:
            nextdigitssame = False
    print('strnextvarsegment ' + strnextvarsegment)
    print('strnextrefsegment ' + strnextrefsegment) 
    print('same segment comfirmation above, same is good')
             
    if isinverted == True and nextdigitssame == True :
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT  !!!!!!!!!!!!!!  ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
        print(variantrule + "      PRESENT                                      ******************")
        #varheadoffset = varheadoffset + difflength ---commented, no change in length of the sequence due to the variation
        #varheadoffsetTemp = min(varheadoffsetTemp, start4digitnumber)
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )
    
    

In [17]:
# sample -- c.0021_0039con0061_0079##LP
def verifyvarient10present(variantrule):
    print("in rule verification 10 ----------------------" + variantrule)
    global varheadoffset
    global offsetapplicablepair
    global var_seq_record
    global ref_seq_record
   
    #c.0021_0039con0061_0079##LP
    
    start4digitnumber = int(variantrule[2])*1000 + int(variantrule[3])*100 + int(variantrule[4])*10 + int(variantrule[5])*1
    print(start4digitnumber)
    end4digitnumber = int(variantrule[7])*1000 + int(variantrule[8])*100 + int(variantrule[9])*10 + int(variantrule[10])*1
    print(end4digitnumber)
    
    '''
    if offsetapplicablepair < start4digitnumber:
        varheadoffsetTemp = varheadoffset
    else:
        varheadoffsetTemp = 0
    '''
    varheadoffsetTemp = getnetoffsetapplicable(start4digitnumber)
    
    start4digitnumbercon = int(variantrule[14])*1000 + int(variantrule[14])*100 + int(variantrule[16])*10 + int(variantrule[17])*1
    print(start4digitnumbercon)
    end4digitnumbercon = int(variantrule[19])*1000 + int(variantrule[20])*100 + int(variantrule[21])*10 + int(variantrule[22])*1
    print(end4digitnumbercon)

    sequencepointer = 25
    strdiagnosis = variantrule[sequencepointer] + variantrule[sequencepointer+1]
    print(strdiagnosis)
    
    print('var head offset 10--' + str(varheadoffsetTemp))
    print('offsetapplicablepair == ' + str(offsetapplicablepair))
    
    print("FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF")
    
    islengthsame = False
    isconverted = True
    nextdigitssame = True
    
    difflength = end4digitnumber - start4digitnumber +1
    print(difflength)
    difflengthcon = end4digitnumbercon - start4digitnumbercon +1
    print(difflengthcon)
    if difflength == difflengthcon:
        islengthsame = True
    
    strvarsegment = ""
    strrefsegment = ""
    for i in range(difflength):
        print('var ' + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i])
        print('refcon ' + ref_seq_record.seq[start4digitnumbercon+i])
        strvarsegment = strvarsegment + var_seq_record.seq[start4digitnumber + varheadoffsetTemp +i]
        strrefsegment = strrefsegment + ref_seq_record.seq[start4digitnumbercon+i]
        if var_seq_record.seq[start4digitnumber +varheadoffsetTemp +i] != ref_seq_record.seq[start4digitnumbercon+i]:
            isconverted = False
    print('strvarsegment ' + strvarsegment)
    print('strrefsegment ' + strrefsegment)
    print('converted comfirmation above, i.e same [var, ref-con] is good')
    
    
    strnextvarsegment = ""
    strnextrefsegment = ""
    for j in range(samenesslookahead):
        print('var next ' + var_seq_record.seq[start4digitnumber+difflength + varheadoffsetTemp +j])
        print('ref next ' + ref_seq_record.seq[end4digitnumber+1+j])
        strnextvarsegment = strnextvarsegment + var_seq_record.seq[start4digitnumber+ difflength + varheadoffsetTemp +j]
        strnextrefsegment = strnextrefsegment + ref_seq_record.seq[end4digitnumber+1+j]
        if var_seq_record.seq[start4digitnumber+difflength +varheadoffsetTemp +j] != ref_seq_record.seq[end4digitnumber+1+j]:
            nextdigitssame = False
    print('strnextvarsegment ' + strnextvarsegment)
    print('strnextrefsegment ' + strnextrefsegment) 
    print('same segment comfirmation above, same is good')
             
    if isconverted == True and nextdigitssame == True and islengthsame == True:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "PRESENT   !!!!!!!!!!!!!! ".ljust(30)+ spcr + "Diagnosis: " + longform(strdiagnosis) )
        print(variantrule + "      PRESENT                                         ******************")
        #varheadoffset = varheadoffset + difflength ---commented, no change in length of the sequence due to the variation
        #varheadoffsetTemp = min(varheadoffsetTemp, start4digitnumber)
    else:
        reportlines.append("VARIANT "+ spcr + variantrule.ljust(40)+ spcr + "Absent".ljust(30)+ spcr + strdiagnone )

In [18]:
import re

def checkvariantexists(variantnumber, variantrule ):
    #print(variantnumber)
    #print(variantrule)
    match = 0
    matchObj1 = re.match( r'c\.\d\d\d\d[a-zA-Z]>[a-zA-Z]##', variantrule)
    if matchObj1:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 1')
        match = 1
        isvarientpresent = verifyvarient1present(variantrule)
        
    matchObj2 = re.match( r'c\.\d\d\d\d_\d\d\d\d[Dd][Ee][Ll]##', variantrule)
    if matchObj2:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 2')
        match = 1
        isvarientpresent = verifyvarient2present(variantrule)
        
    matchObj3 = re.match( r'c\.\d\d\d\d_\d\d\d\d[Dd][Ee][Ll][atgcATGC]+##', variantrule)
    if matchObj3:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 3')
        match = 1
        isvarientpresent = verifyvarient3present(variantrule)
        
    matchObj4 = re.match( r'c\.\d\d\d\d_\d\d\d\d[Dd][Uu][Pp]##', variantrule)
    if matchObj4:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 4')
        match = 1
        isvarientpresent = verifyvarient4present(variantrule)
        
    matchObj5 = re.match( r'c\.\d\d\d\d_\d\d\d\d[Dd][Uu][Pp][atgcATGC]+##', variantrule)
    if matchObj5:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 5')
        match = 1
        isvarientpresent = verifyvarient5present(variantrule)
        
    matchObj6 = re.match( r'c\.\d\d\d\d_\d\d\d\d[Ii][Nn][Ss][atgcATGC]+##', variantrule)
    if matchObj6:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 6')
        match = 1
        isvarientpresent = verifyvarient6present(variantrule)
        
    matchObj7 = re.match( r'c\.\d\d\d\d_\d\d\d\d[Dd][Ee][Ll][Ii][Nn][Ss][atgcATGC]+##', variantrule)
    if matchObj7:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 7')
        match = 1
        isvarientpresent = verifyvarient7present(variantrule)
        
    matchObj8 = re.match( r'c\.\d\d\d\d_\d\d\d\d[Dd][Ee][Ll][atgcATGC]+[Ii][Nn][Ss][atgcATGC]+##', variantrule)
    if matchObj8:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 8')
        match = 1
        isvarientpresent = verifyvarient8present(variantrule)
    
    #0021_0039inv##PP
    matchObj9 = re.match( r'c\.\d\d\d\d_\d\d\d\d[Ii][Nn][Vv]##', variantrule)
    if matchObj9:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 9')
        match = 1
        isvarientpresent = verifyvarient9present(variantrule)
        
    matchObj10 = re.match( r'c\.\d\d\d\d_\d\d\d\d[Cc][Oo][Nn]\d\d\d\d_\d\d\d\d##', variantrule)
    if matchObj10:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- matched 10')
        match = 1
        isvarientpresent = verifyvarient10present(variantrule)
        
    if match == 0:
        print('\n\n Variant#' + str(variantnumber)+' Variant:'+variantrule + ' --- NO MATCHING RULE FOUND!!!!!')

In [19]:
'''
print(var_seq_record.seq[:100])
print(ref_seq_record.seq[:100])
'''

'\nprint(var_seq_record.seq[:100])\nprint(ref_seq_record.seq[:100])\n'

In [20]:
import re
netdeletes=0
netinserts=0
netduplicates=0

global var_seq_record
global ref_seq_record

file1 = open('BRCA1-VARIANTS.txt', 'r') 
Variants = file1.readlines() 

for root, dirs, files in os.walk(dirpath):
    for filename in files:
            print(filename)
            testfilecount+=1
            testfilenames.append(filename)

print(testfilecount)
for i in range(testfilecount):
    print(str(i) + " -- " + testfilenames[i])
    var_seq_record = SeqIO.read(dirpath+testfilenames[i], "fasta")
    reportlines.append("--------------------------------------------------------------------------------------------------------------------------")
    reportlines.append("For file:   " + testfilenames[i])
    reportlines.append(var_seq_record.seq[:20]+'|'+var_seq_record.seq[20:40]+'|'+var_seq_record.seq[40:60]+'|'+var_seq_record.seq[60:80]+'|'+var_seq_record.seq[80:100])
    reportlines.append(ref_seq_record.seq[:20]+'|'+ref_seq_record.seq[20:40]+'|'+ref_seq_record.seq[40:60]+'|'+ref_seq_record.seq[60:80]+'|'+ref_seq_record.seq[80:100])
    reportlines.append("--------------------------------------------------------------------------------------------------------------------------")
    varheadoffset = 0
    lstvarhits = []
    count = 0
    # Strips the newline character 
    for variant in Variants: 
        #print("Line {}: {}".format(count+1, variant.strip()))
        checkvariantexists(count+1, variant.strip() )
        count+=1
        
reportlines.append("--------------------------------------------------------------------------------------------------------------------------")
reportlines.append("===============================================END OF GENOMICS REPORT=====================================================")

P004-BRCA1-CV01.fasta
P004-BRCA1-CV02.fasta
2
0 -- P004-BRCA1-CV01.fasta


 Variant#1 Variant:c.0009_0012del##BB --- matched 2
in rule verification 2----------------------c.0009_0012del##BB
9
12
 in getnetoffsetapplicable function
netoffset total =  0
BB
var head offset 2--0
offsetapplicablepair == 10000000
4
var T
ref T
var G
ref C
var C
ref C
var T
ref T
strvarsegment TGCT
strrefsegment TCCT
deletion comfirmation above, NOT same is good
var next T
ref next G
var next G
ref next G
strnextvarsegment TG
strnextrefsegment GG
same segment confirmation above, same is good


 Variant#2 Variant:c.0009_0012deqlTCCT##BB --- NO MATCHING RULE FOUND!!!!!


 Variant#3 Variant:c.0010C>G##LB --- matched 1
in rule verification 1----------------------c.0010C>G##LB
10
 in getnetoffsetapplicable function
netoffset total =  0
C
G
substitution confirmation 1
LB
var head offset 1--0
offsetapplicablepair == 10000000
G
C
substitution confirmation 2
c.0010C>G##LB      PRESENT                               ***

In [21]:
for i in range(len(reportlines)):
        print(reportlines[i])

                           KAIGENIX CLINICAL MANAGEMENT - GENOMICS DEPT.                            
                            Genetic Variations Report (for BRCA1 gene)
                             
Details:
--------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------
For file:   P004-BRCA1-CV01.fasta
GCTGAGACTTGCTGGACGGG|GACAGCCTGTCGGGGTTTCT|CTCAGGAGGCCGCCCCTGCG|CTCAGGAGGCCTTCACCCTC|TGCTCTGGGTAAAGTTCATT
GCTGAGACTTCCTGGACGGG|GGACAGGCTGTGGGGTTTCT|CAGATAACTGGGCCCCTGCG|CTCAGGAGGCCTTCACCCTC|TGCTCTGGGTAAAGTTCATT
--------------------------------------------------------------------------------------------------------------------------
VARIANT |->| c.0009_0012del##BB                      |->| Absent                        |->| ---
VARIANT |->| c.0010C>G##LB                           |->| PRESENT  !!!!!!!!!!!!

In [22]:
import datetime
currenttime = datetime.datetime.now()

newfilename='genomicsreportfile'+'-'+str(currenttime.year)+'-'+str(currenttime.month)+'-'+str(currenttime.day)+'-'+str(currenttime.hour)\
    +'-'+str(currenttime.minute)+'-'+str(currenttime.second)+'-'+str(currenttime.microsecond)+'.txt'
fileobject = open(newfilename, 'w')
for i in range(len(reportlines)):
    fileobject.write(str(reportlines[i]))
    fileobject.write('\n')
fileobject.close()

In [23]:
from fpdf import FPDF 
import datetime 
pdf = FPDF()     
pdf.add_page() 
pdf.set_font("Arial", size = 8) 
currenttime = datetime.datetime.now()
strpdffilename='genomicsreportfile'+'-'+str(currenttime.year)+'-'+str(currenttime.month)+'-'+str(currenttime.day)+'-'+str(currenttime.hour)\
    +'-'+str(currenttime.minute)+'-'+str(currenttime.second)+'-'+str(currenttime.microsecond)+'.pdf'

for i in range(len(reportlines)):
    #fileobject.write(str(reportlines[i]))
    pdf.cell(200, 10, txt = str(reportlines[i]), ln = i+1, align = 'L')
    
# save the pdf with name .pdf 
pdf.output(strpdffilename)    

''